In [ ]:
%%html
<style>
table {margin-left: 0 !important;}
table td, table th, table tr {text-align:left !important;}
</style>

<div>
<img src="attachment:Bovag.jpg" width="400">
</div>

# Multiple Binomial Logistic Regression Models - Groepsopdracht voor werkcollege

## Casus
De Hogeschool Utrecht (HU) heeft zo’n 4.000 werknemers in dienst. Elk jaar vertrekt echter zo’n 15% van de werknemers en de corporate HRM afdeling heeft de taak om deze werknemers te vervangen. Het kost veel tijd en geld om geschikte werknemers te vinden, omdat er sprake is van een krappe arbeidsmarkt (er zijn meer vacatures dan werkzoekenden).

Daarom wil de HU investeren in het *behouden* van werknemers, om zo het vertrekpercentage te verlagen. Het management heeft daarom aan het People Analytics team gevraagd te onderzoeken welke factoren het vertrek van werknemers kunnen verklaren, zodat gerichte interventies kunnen worden ingezet om werknemers te behouden. Jullie werken voor het People Analytics team en gaan onderzoeken welke factoren vertrek voorspellen. Dat doen jullie op basis van een dataset met daarin 4.410 (oud-)werknemers van de HU.

Jullie gaan een **hiërarchisch regressiemodel** bouwen en passen daarbij **het principe van parsimony** toe om te komen tot het meest simpele voorspellend model. Op basis van eerdere onderzoeken naar oorzaken van vertrek bouwen jullie het multiple regressie model in drie stappen op (of: 3 modellen). Hieronder staat toegelicht welke predictoren in elk model worden toegevoegd.  

Vertrek (Y) wordt gemeten als *de medewerker is in het afgelopen jaar vertrokken (ja/nee)*, heeft als variabelenaam ***Attrition_rec*** en is van *nominaal* meetniveau.

In onderstaande tabel zie je welke predictoren (X) je in welk model moet opnemen:


| Model | Predictor (X) | Variabelenaam | Meetniveau |
| :--- | :--- | :--- | :--- |
| 1 | Leeftijd | Age | Ratio |
| 1 | Opleidingsniveau | Education | Ordinaal |
| 1 | Geslacht | Gender | Nominaal |
| 1 | Aantal organisaties waarvoor de werknemer gewerkt heeft | NumCompaniesWorked | Ratio |
| 1 | Jaren werkervaring | TotalWorkingYears | Ratio |
| 2 | Maandsalaris | MonthlyIncome| Ratio |
| 2 | Reisafstand in kilometers | DistanceFromHome | Ratio |
| 2 | Jaren sinds laatste promotie | YearsSinceLastPromotion | Ratio |
| 2 | Performancescore | PerformanceRating | Interval |
| 2 | Aantal jaren werkzaam onder huidige manager | YearsWithCurrManager | Ratio |
| 3 | Werktevredenheid | JobSatisfaction | Interval |
| 3 | Werk-privé balans | WorkLifeBalance | Interval |
| 3 | Werkbetrokkenheid | JobInvolvement | Interval |

**Voetnoten bij tabel**
1. *Maandsalaris*: Het maandsalaris wordt weergegeven in roepies (Indiase munteendheid). Dat is natuurlijk raar, wij als HU-docenten krijgen gewoon in euro’s uitbetaald. We hadden een creatieve reden kunnen bedenken waarom hier roepies staan, maar jullie snappen de echte reden vast wel: het is een fictief bestand, gemaakt door Indiase HRM docent, waarvan wij met toestemming dankbaar gebruik van maken voor onze eigen fictieve casus.
2. *Interval variabelen*: Deze schalen zijn eigenlijk van ordinaal meetniveau. In de praktijk worden schalen van ordinaal meetniveau met een structuur die lijkt op een Likert-schaal vaak behandeld als schalen van interval niveau. Dat doen wij hier ook. Daarop is uiteraard kritiek te leveren, maar die negeren wij voor deze opdracht voor ons gemak even.

Run eerst onderstaande cel om de benodigde packages te laden.

In [ ]:
# Als je een foutmelding krijgt, uncomment dan de volgende X regels (verwijder '#') en run de cel opnieuw.

# !pip install numpy==1.23.5
# !pip install pandas==1.5.3
# !pip install statsmodels==0.14.0

import numpy as np
import pandas as pd
import statsmodels.api as sm

print(np.__version__,pd.__version__,sm.__version__)

Laad nu de data in door de volgende cel te runnen

In [ ]:
hrdata = pd.read_csv('https://raw.githubusercontent.com/uashogeschoolutrecht/ADDB-DE-22/master/WC5_binomial_logistic_regression_model/General_data.csv',sep=';')
hrdata.head()

*   We kiezen ervoor om rijen met ontbrekende kenmerkwaarden te verwijderen.
* Daarnaast passen we de waarden (Yes/No) van de afhankelijke variabele aan naar binaire waarden (1/0). 
* We zorgen dat het type van een aantal kenmerken als numeriek (integer) wordt herkend.
* Tenslotte voegen we een constante toe, wat nodig is voor het trainen van de modellen met het Python-package statsmodels.

In [ ]:
hrdata.replace(' ', np.nan, inplace=True)
hrdata = hrdata.dropna()

hrdata['Attrition'].replace(['Yes','No'],[1,0],inplace=True)

hrdata = sm.tools.tools.add_constant(hrdata, prepend=True, has_constant='add')

hrdata = hrdata.astype({'TotalWorkingYears': 'int', 'NumCompaniesWorked': 'int', 'JobSatisfaction': 'int', 'WorkLifeBalance': 'int'})


## Stap 1: Dummy variabelen maken

In het eerste model worden de variabelen Education en Gender toegevoegd. Dit zijn categorische variabelen waarvan eerst dummy variabelen worden gemaakt. 
- Maak de dummy variabelen aan met Education type ‘Bachelor’ (waarde 1) als baseline categorie. Oftewel, maak vier dummy variabelen aan: Below_college, College, Master en Doctor. 
- Maak de dummy variabelen aan voor vrouwen, genaamd Female. (De baseline wordt dus gevormd door de waarde Male.) 
- Voeg de nieuwe dummy-kenmerken toe aan de tabel en verwijder de originele variabelen.  
  

In [ ]:
education_dummies = pd.get_dummies(hrdata['Education'],prefix = 'Education', prefix_sep = '.')
education_dummies = education_dummies.drop('Education.1', axis = 1)

In [ ]:
gender_dummies = pd.get_dummies(hrdata['Gender'],prefix = 'Gender', prefix_sep = '')
gender_dummies = gender_dummies.drop('GenderMale', axis = 1)

In [ ]:
hrdata = pd.concat([hrdata,education_dummies,gender_dummies], axis = 1)
hrdata = hrdata.drop(['Education','Gender'],axis = 1)

## Stap 2 - Multiple Binomial Logistic Regression Models

Voer de multiple binomial logistic regression models uit in R en beantwoord de vragen. Je moet hiervoor alle onderstaande codes runnen.

**Hieronder staat de syntax voor het runnen van Model 1:**

In [ ]:
model1 = sm.Logit(hrdata[['Attrition']], hrdata[['const','Age','Education.2','Education.3','Education.4','Education.5', 
                                                 'GenderFemale','NumCompaniesWorked','TotalWorkingYears']])
result1 = model1.fit(method='newton')
display(result1.summary())

**Hieronder staat de syntax voor het runnen van Model 2:**

In [ ]:
model2 = sm.Logit(hrdata[['Attrition']], hrdata[['const','Age','Education.2','Education.3','Education.4','Education.5', 
                                                 'GenderFemale','NumCompaniesWorked','TotalWorkingYears', 'MonthlyIncome',
                                                 'DistanceFromHome','YearsSinceLastPromotion','PerformanceRating',
                                                 'YearsWithCurrManager']])
result2 = model2.fit(method='newton')
display(result2.summary())

**Hieronder staat de syntax voor het runnen van Model 3:**

In [ ]:
model3 = sm.Logit(hrdata[['Attrition']], hrdata[['const','Age','Education.2','Education.3','Education.4','Education.5', 
                                                 'GenderFemale','NumCompaniesWorked','TotalWorkingYears', 'MonthlyIncome',
                                                 'DistanceFromHome','YearsSinceLastPromotion','PerformanceRating',
                                                 'YearsWithCurrManager','JobSatisfaction','WorkLifeBalance','JobInvolvement']])
result3 = model3.fit(method='newton')
display(result3.summary())

**Let op:** Voor model 3 vragen we ook de Odds ratio's en bijbehorende confidence intervals op. Deze gaan we inhoudelijk interpreteren.

In [ ]:
params = result3.params
conf = result3.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(conf))

### Vraag 1
*Interpreteer de pseudo R squares en concludeer welk model het beste het vertrek van werknemers voorspelt én hoe goed dit model het vertrek van werknemers voorspelt.*

Typ hier je antwoord

### Vraag 2
*Interpreteer de log-likelihoods m.b.t. verbetering ten opzichte van vorige modellen en trek een passende conclusie over de model fit.*

Typ hier je antwoord

### Vraag 3
*Interpreteer voor model 3 per predictor, indien relevant, de odds ratio en significantie (o.b.v. de confidence intervals) zeer nauwkeurig.*

Typ hier je antwoord

## Stap 3 - Model Parsimony

- Verwijder alle *niet significante predictoren* uit model 3 en run het model opnieuw.
- Bereken vervolgens de AIC voor model 3 met álle predictoren *en* de AIC voor model 3 met álleen significante predictoren.
- Beantwoord daarna onderstaande vragen.   

In [ ]:
# model 4 wordt berekend met enkel de significante predictoren

model4 = sm.Logit(hrdata[['Attrition']], hrdata[['const','Age','NumCompaniesWorked','TotalWorkingYears', 'MonthlyIncome','YearsSinceLastPromotion','YearsWithCurrManager','JobSatisfaction','WorkLifeBalance']])
result4 = model4.fit(method='newton')

print("Akaike Information Criterion: {}, {}".format(round(result3.aic,2),round(result4.aic,2))) 


### Vraag 1
*Leidt het verwijderen van niet significante predictoren daadwerkelijk tot een meer ‘parsimonious’ model? Licht je antwoord toe.*

Typ hier je antwoord

## Stap 4 - Conclusies en advies

*Trek conclusies en breng op basis van bovenstaande resultaten een advies uit aan de opdrachtgevers van de HU.*

Zorg ervoor dat je conclusies en advies aan de volgende criteria voldoen:
- Conclusies: Geef een korte beschrijving van resultaten in **‘lekentaal’** (te begrijpen voor iemand zonder veel kennis van statistiek) en verwerk daarin ook de verklaarde variantie. Betrek hierin ook de model parsimomy.
- Advies: Geef concrete aanbevelingen. Schrijf bijvoorbeeld niet alleen op ‘er is vervolgonderzoek nodig’, of ‘interventies moeten gericht zijn op variabele Y’, maar geef ook concrete suggesties.

Typ hier jullie conclusies in lekentaal

Typ hier jullie advies